# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [101]:
import sqlite3    # Python interface for SQLite databases

In [102]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [103]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
dummy:
  0: DEGRE(NUM)
lieu:
  0: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
soldwine:
  0: NV(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NB(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)


From this point we will use __%%sql__ magic

In [104]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [105]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [106]:
%%sql SELECT *
FROM Master1
WHERE NP = 2;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
49,Morgon,12,1984,150,2,Boxler,Albert,Alsace
50,Chenas,11.5,1984,100,2,Boxler,Albert,Alsace
54,Beaujolais Villages,10.5,1976,900,2,Boxler,Albert,Alsace


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

In [107]:
%%sql SELECT *
FROM Master1
WHERE NV = 1;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
1,Mercurey,11.5,1980,300,1,Bohn,Rene,Alsace
1,Mercurey,11.5,1980,1,73,Bourguignon,Emile,Jura


In [108]:
%%sql SELECT *
FROM Master1
WHERE NP = 1;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
1,Mercurey,11.5,1980,300,1,Bohn,Rene,Alsace
3,Savigny les Beaunes,12.1,1978,400,1,Bohn,Rene,Alsace
54,Beaujolais Villages,10.5,1976,500,1,Bohn,Rene,Alsace


Answer : From the two tables above we can see that some of the main functional dependencies can be summarized as per below:

1) NV-> CRU, DEGRE, MILL
2) NP -> NOM, PRENOM, REGION

Solution : There are partial and transitive relationship that need to be remove in order to normalize this table.
So, first we need to separate relationship from partial keys and other attributes (2NF)
And secondly, we need to take care of transitive dependencies between non-key attributes (3NF) and seperates them into a different tables. 



#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [109]:
%%sql select *
from master2
WHERE NV = 1


 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
1,Mercurey,11.5,1980,1977-11-02,BORDEAUX,33,2,Artaud,Antonin,moyen,NOUVELLE-AQUITAINE
1,Mercurey,11.5,1980,2015-10-16,PARIS,1,44,Gide,Andre,petit,ÎLE-DE-FRANCE
1,Mercurey,11.5,1980,1983-12-31,RENNES,1,45,Giono,Jean,petit,BRETAGNE


Main functional dependencies =
1) NV-> CRU, DEGREE, MILL
2) NB -> NOM, PRENOM, TYPE
3) LIEU -> REGION 

We then need to seperate the aforementionned dependencies into 3 seperates tables. 
1) Vin
2) Buveur
3) Region


Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [177]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [178]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
lieu:
  0: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
soldwine:
  0: NV(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NB(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)


In [176]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [175]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [114]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
(sqlite3.OperationalError) database table is locked [SQL: 'DROP TABLE IF EXISTS dummy;'] (Background on this error at: http://sqlalche.me/e/e3q8)


#### Exercise 1.3

Create the new tables from Master1:

In [158]:
%%sql DROP TABLE IF EXISTS wine;

CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE 
FROM MASTER1

 * sqlite:///wine.db
Done.
Done.


[]

In [159]:
%%sql DROP TABLE IF EXISTS lieu;

CREATE TABLE lieu AS
SELECT REGION
FROM MASTER1

 * sqlite:///wine.db
Done.
Done.


[]

In [160]:
%%sql DROP TABLE IF EXISTS production;
CREATE TABLE production AS
SELECT NV, NP, QTE
FROM MASTER1

 * sqlite:///wine.db
Done.
Done.


[]

In [161]:
%%sql DROP TABLE IF EXISTS producer;
CREATE TABLE producer AS
SELECT NP, NOM, PRENOM, REGION
FROM MASTER1

 * sqlite:///wine.db
Done.
Done.


[]

In [127]:
%sql DROP TABLE IF EXISTS client;

 * sqlite:///wine.db
Done.


[]

In [162]:
%%sql DROP TABLE IF EXISTS client;
CREATE TABLE client AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2

 * sqlite:///wine.db
Done.
Done.


[]

In [ ]:
%%sql 
CREATE TABLE client AS
SELECT NB, NOM, PRENOM, TYPE
FROM MASTER2

#### Exercise 1.4

Create the new tables from Master2:

In [163]:
%%sql DROP TABLE IF EXISTS soldwine;
CREATE TABLE soldwine AS
SELECT NV, DATES, LIEU, QTE, NB
FROM MASTER2

 * sqlite:///wine.db
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [164]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
lieu:
  0: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
soldwine:
  0: NV(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NB(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)


In [183]:
%%sql
SELECT TYPE
FROM client 


 * sqlite:///wine.db
Done.


TYPE
petit
moyen
moyen
gros
moyen
petit
petit
petit
gros
petit


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [182]:
%%sql
SELECT DISTINCT CRU, REGION 
FROM wine INNER JOIN production ON (wine.NV=production.NV) 
INNER JOIN producer ON (production.NP=producer.NP)
WHERE cru IN ('Pommard', 'Brouilly')


 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

#### Exercise 2.6

Which producers have not produced any wine?

#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

#### Exercise 2.10

List all wines from 1980

#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?